<a href="https://colab.research.google.com/github/pvpogorelova/metrics_24_25/blob/main/sem_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Модель упорядоченного выбора (ordered logit).**

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

Датасет **ologit.dta** содержит данные социологического исследования, изучающего факторы, влияющие на вероятность поступления абитуриентов в высшие учебные заведения.

Объём выборки составляет 400 наблюдений. Основная зависимая переменная (apply) является *упорядоченной категориальной*, что делает целесообразным применение моделей порядковой логистической регрессии (Ordered Logit).

**Цель исследования**

Анализ направлен на оценку влияния:

    * уровня образования родителей (pared),
    * типа школы (public),
    * академической успеваемости (gpa)
на вероятность подачи заявки в высшие учебные заведения.

In [2]:
url = "https://stats.idre.ucla.edu/stat/data/ologit.dta"
data = pd.read_stata(url)

data.head(5)

,apply,pared,public,gpa
0,very likely,0,0,3.26
1,somewhat likely,1,0,3.21
2,unlikely,1,1,3.94
3,somewhat likely,0,0,2.81
4,somewhat likely,0,0,2.53


In [4]:
# Зависимая переменная 'apply' (категориальная) (3 уровня: "unlikely", "somewhat likely", "very likely")
y = data['apply'].astype('category').cat.reorder_categories(
    ["unlikely", "somewhat likely", "very likely"], ordered=False
)

# Независимые переменные: 'pared', 'public', 'gpa'
X = data[['pared', 'public', 'gpa']]

In [5]:
# Оценим logit-модель упорядоченного выбора
model = OrderedModel(y, X, distr='logit')
result = model.fit(method='bfgs', disp=False)
print(result.summary())

                             OrderedModel Results                             
Dep. Variable:                  apply   Log-Likelihood:                -358.51
Model:                   OrderedModel   AIC:                             727.0
Method:            Maximum Likelihood   BIC:                             747.0
Date:                Sun, 20 Apr 2025                                         
Time:                        15:48:36                                         
No. Observations:                 400                                         
Df Residuals:                     395                                         
Df Model:                           3                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
pared                           1.0476      0.266      3.942      0.000       0.527       1.569
p

/usr/local/lib/python3.11/dist-packages/statsmodels/miscmodels/ordinal_model.py:205: Warning: the endog has ordered == False, risk of capturing a wrong order for the categories. ordered == True preferred.
  warnings.warn("the endog has ordered == False, "


Рассчитаем отношение шансов.

In [6]:
# Отношения шансов (OR) = exp(coef)
odds_ratios = pd.DataFrame({
    'OR': np.exp(result.params),
    'P>|z|': result.pvalues,
    'CI 2.5%': np.exp(result.conf_int()[0]),
    'CI 97.5%': np.exp(result.conf_int()[1]),
})
print("\nOdds Ratios (OR):")
print(odds_ratios)


Odds Ratios (OR):
                                   OR         P>|z|   CI 2.5%   CI 97.5%
pared                        2.850901  8.093660e-05  1.693333   4.799784
public                       0.943077  8.440140e-01  0.526030   1.690768
gpa                          1.851167  1.813824e-02  1.110697   3.085287
unlikely/somewhat likely     9.057094  4.702483e-03  1.965374  41.738085
somewhat likely/very likely  2.095426  2.551448e-20  1.791009   2.451586


Интерпретация предельных эффектов на примере переменной **gpa**:

при увеличении среднего балла (gpa) на одну единицу шансы попадания в категорию "очень вероятно" (very likely) по сравнению с объединёнными категориями "маловероятно" и "отчасти вероятно" (unlikely + somewhat likely) возрастают в 1.85 раза при фиксированных остальных переменных.

Если предположению о пропорциональности шансов выполнено, то такое же увеличение (в 1.85 раза) наблюдается между категорией "маловероятно" (unlikely) и объединёнными категориями "отчасти вероятно" и "очень вероятно" (somewhat likely + very likely).

Для модели упорядоченного выбора должно выполняться предположение о пропорциональности шансов (parallel regression assumption). Оно значит следующее: **коэффициенты модели упорядоченного выбора одинаковы для всех границ категорий зависимой переменной (влияние предикторов на все возможные переходы между категориями одинаково).**

Для проверки используется тест Бранта. В Python отсутствует реализация теста Бранта. Однако его можно проделать вручную.

**Алгоритм:**
* Оцениваем серию бинарных логистических регрессий для каждой границы категорий
зависимой переменной.
* Сравниваем коэффициенты моделей между собой. Если они статистически не различаются — предположение выполняется.
* Используем тест Вальда (Wald test) или критерий отношения правдоподобий (Likelihood Ratio).

In [8]:
# Тест Бранта (ручная реализация через тест Вальда)
# 1. Разбиваем зависимую переменную на бинарные категории
# Для 'apply':
#   - y1 = 1 если apply > "unlikely" (т.е. "somewhat likely" или "very likely")
#   - y2 = 1 если apply > "somewhat likely" (т.е. "very likely")
data['y1'] = (data['apply'].cat.codes > 0).astype(int)  # сравниваем: unlikely vs (somewhat + very)
data['y2'] = (data['apply'].cat.codes > 1).astype(int)  # сравниваем: (unlikely + somewhat) vs very

In [9]:
from statsmodels.formula.api import logit
# 2. Оцениваем две бинарные логистические регрессии
model1 = logit('y1 ~ pared + public + gpa', data=data).fit(disp=0)
model2 = logit('y2 ~ pared + public + gpa', data=data).fit(disp=0)

In [10]:
# 3. Тест Вальда на равенство коэффициентов (аналог теста Бранта)
from scipy.stats import chi2
def brant_test(model1, model2):
    delta = model1.params - model2.params
    cov = model1.cov_params() + model2.cov_params()  # упрощенное предположение
    wald_stat = delta.T @ np.linalg.inv(cov) @ delta
    p_value = chi2.sf(wald_stat, df=len(delta))
    return wald_stat, p_value

wald_stat, p_value = brant_test(model1, model2)
print(f"Brant Test (Wald statistic): {wald_stat:.4f}, p-value: {p_value:.4f}")

Brant Test (Wald statistic): 107.0425, p-value: 0.0000


Так как p-value < 0.05, то гипотеза $H_1$ не отвергается, то есть предположение о пропорциональных шансах нарушено. Возможно, стоит использовать Generalized Ordered Logit (непараметрическую версию).

In [12]:
# Получаем предсказанные вероятности исходов для всех наблюдений
pred_probs = result.model.predict(result.params, result.model.exog)
pred_probs

array([[0.54884071, 0.35932276, 0.09183653],
       [0.30558191, 0.47594216, 0.21847593],
       [0.22938356, 0.47819057, 0.29242587],
       ...,
       [0.69380357, 0.25470075, 0.05149568],
       [0.54884071, 0.35932276, 0.09183653],
       [0.50896793, 0.38494062, 0.10609145]])

Теперь давайте рассчитаем предельные эффекты на вероятность выбора каждой из категорий.

In [13]:
# Категории зависимой переменной
categories = ['unlikely', 'somewhat likely', 'very likely']

# Функция для расчета предельных эффектов
def calculate_margeff(model, result, delta=1e-5):
    n_vars = model.exog.shape[1]  # Число регрессоров
    n_cats = len(categories)
    margeff = np.zeros((n_vars, n_cats))

    # Базовые вероятности
    base_probs = result.predict()

    for i in range(n_vars):
        # Создаем измененные данные
        X_high = model.exog.copy()
        X_high[:, i] += delta

        # Новые вероятности
        probs_high = result.model.predict(result.params, X_high)

        # Предельный эффект
        margeff[i] = (probs_high - base_probs).mean(axis=0) / delta

    return margeff

# Расчет и вывод результатов
margeff = calculate_margeff(model, result)
margeff

array([[-0.24536074,  0.1537694 ,  0.09159134],
       [ 0.01372596, -0.00860314, -0.00512283],
       [-0.14439455,  0.09050476,  0.05388979]])